<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания  10


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Invoice в C#, который будет представлять информацию о
фактурах за поставленные товары или оказанные услуги. На основе этого класса
разработать 2-3 производных класса, демонстрирующих принципы наследования и
полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и
методы, а также переопределены некоторые методы базового класса для
демонстрации полиморфизма.

Требования к базовому классу Invoice:

• Атрибуты: Номер фактуры (InvoiceNumber), Дата выдачи (IssueDate), Общая
сумма (TotalAmount).

• Методы:
1. CalculateTotal(): метод для расчета общей суммы по фактуре.
2. AddLine(LineItem lineItem): метод для добавления позиции в фактуру.
3. RemoveLine(LineItem lineItem): метод для удаления позиции из
фактуры.

Требования к производным классам:
1. ТоварнаяФактура (GoodsInvoice): Должна содержать дополнительные
атрибуты, такие как Дата поставки (SupplyDate). Метод AddLine() должен
быть переопределен для добавления информации о дате поставки товара
при добавлении позиции.
2. УслуговаяФактура (ServiceInvoice): Должна содержать дополнительные
атрибуты, такие как Дата оказания услуги (ServiceDate).
Метод RemoveLine() должен быть переопределен для добавления
информации о причине аннулирования услуги при удалении позиции.
3. КомбинированнаяФактура (CombinedInvoice) (если требуется третий класс):
Должна содержать дополнительные атрибуты, такие как Наличие возврата
(ReturnAllowed). Метод CalculateTotal() должен быть переопределен для
учета возможного возврата товара или услуги при расчете общей суммы.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [2]:
using System;
using System.Collections.Generic; 

/*
Это стандартная библиотека в C#, которая содержит коллекции данных (контейнеры) для работы с элементами разных типов. Основные коллекции, которые предоставляет это пространство имен, — это обобщенные (generic) коллекции, такие как:

List<T> — динамический список, который может хранить элементы любого типа T. Например: List<int>, List<string>.
Dictionary<TKey, TValue> — коллекция пар "ключ-значение".
Queue<T> — очередь (FIFO — first in, first out, "первый вошел, первый вышел").
Stack<T> — стек (LIFO — last in, first out, "последний вошел, первый вышел").----------------------------------------------------------------
*/


// Класс для описания позиции в фактуре
class LineItem
{
    public string Description { get; set; }   // Описание товара или услуги СВОЙСТВО    get — это аксессор (доступник), который возвращает значение свойства. То есть, ты можешь "получить" значение свойства.
                                                                                    //  set — это аксессор, который устанавливает значение свойства. То есть, ты можешь "установить" или изменить значение.
    public decimal Price { get; set; }        // Цена
    public int Quantity { get; set; }         // Количество

    /* это конструктор класса LineItem 
    Конструктор — это специальный метод, который вызывается при создании объекта класса, и его задача — инициализировать объект (задать начальные значения его свойств).
    */
    public LineItem(string description, decimal price, int quantity) 

    {
        Description = description;
        Price = price;
        Quantity = quantity;
    }

    // Публичный метод 
    public decimal GetTotal()
    {
        return Price * Quantity;
    }
}

// Базовый класс для всех типов фактур
abstract class Invoice
{
    public string InvoiceNumber { get; set; }
    public DateTime IssueDate { get; set; }
    public decimal TotalAmount { get; protected set; }

    protected List<LineItem> lineItems;

    // Конструктор
    public Invoice(string invoiceNumber, DateTime issueDate)
    {
        InvoiceNumber = invoiceNumber;
        IssueDate = issueDate;
        lineItems = new List<LineItem>(); // создает новый экземпляр списка объектов типа LineItem
    }

    // Метод для расчета общей суммы
    public virtual void CalculateTotal()
    {
        TotalAmount = 0;
        foreach (var item in lineItems)
        {
            TotalAmount += item.GetTotal();
        }
    }

    // Метод для добавления позиции
    public virtual void AddLine(LineItem lineItem)
    {   
        lineItems.Add(lineItem);
    }

    // Метод для удаления позиции
    public virtual void RemoveLine(LineItem lineItem)
    {
        lineItems.Remove(lineItem);
        CalculateTotal();
    }
}

// GoodsInvoice - дочерний класс, который наследует ФУНКЦИОНАЛ от базового(родительского) класса Invoice
class GoodsInvoice : Invoice
{
    public DateTime SupplyDate { get; set; } // объявляет свойство SupplyDate в классе GoodsInvoice, которое будет хранить дату поставки товара.

        /*GoodsInvoice вызывает конструктор базового класса Invoice с двумя параметрами (invoiceNumber и issueDate)
        base() — это специальный вызов конструктора базового класса. 
        Это позволяет инициализировать свойства и выполнить логику базового класса, перед тем как выполнять инициализацию специфичных для GoodsInvoice частей.*/
    public GoodsInvoice(string invoiceNumber, DateTime issueDate, DateTime supplyDate)
        : base(invoiceNumber, issueDate)    
    {
        /* После вызова конструктора базового класса, конструктор GoodsInvoice устанавливает значение свойства SupplyDate на переданное значение supplyDate.
        Это назначает дату поставки товару, используя значение, переданное в конструктор.*/
        SupplyDate = supplyDate; 
    }

    // Переопределение метода AddLine для изменения или расширения поведения базового метода в ДОЧЕРНЕМ классе. override
    public override void AddLine(LineItem lineItem)
    {
        base.AddLine(lineItem);
        base.CalculateTotal();
        /*
        Этот вызов выполняет код базового метода AddLine из класса Invoice.
        Это позволяет использовать логику базового класса для добавления строки (lineItem) в список lineItems и пересчета общей суммы (если таковой имеется).
        Этот вызов необходим, чтобы сохранить стандартное поведение метода, а затем добавить специфичное поведение в классе GoodsInvoice.
        */
        Console.WriteLine($"Товар добавлен с датой поставки: {SupplyDate.ToShortDateString()}");
    }
}
class ServiceInvoice : Invoice
{
    public DateTime ServiceDate { get; set; }

    public ServiceInvoice(string invoiceNumber, DateTime issueDate, DateTime serviceDate)
        : base(invoiceNumber, issueDate)
    {
        ServiceDate = serviceDate;
    }


    public override void AddLine(LineItem lineItem)
    {
        base.AddLine(lineItem);
        Console.WriteLine($"\n\n\n\nДата оказания услуги: {ServiceDate.ToShortDateString()}");
    }


    // Переопределение метода RemoveLine
    public override void RemoveLine(LineItem lineItem)
    {
        base.RemoveLine(lineItem);
        /*
        Этот вызов выполняет код базового метода RemoveLine из класса Invoice.
        Это позволяет использовать логику базового класса для удаления строки (lineItem) в списке lineItems и пересчета общей суммы (если таковой имеется).
        Этот вызов необходим, чтобы сохранить стандартное поведение метода, а затем добавить специфичное поведение в классе GoodsInvoice.
        */
        Console.WriteLine($"Услуга: {lineItem.Description} была аннулирована.\nПричина: ошибка заказа.");
    }
}
class CombinedInvoice : Invoice
{
    /*
        public bool ReturnAllowed { get; set; } — это свойство, которое указывает, разрешены ли возвраты в рамках данной фактуры.
        Тип данных bool означает, что свойство может быть либо true (возвраты разрешены), либо false (возвраты запрещены).
    */
    public bool ReturnAllowed { get; set; }

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed)
        : base(invoiceNumber, issueDate)
    {
        ReturnAllowed = returnAllowed;
    }

    // Переопределение метода CalculateTotal с учетом возвратов
    public override void CalculateTotal()
    {
        base.CalculateTotal();
        if (ReturnAllowed)
        {
            
            TotalAmount -= 50; // Скидка или учет возврата
            Console.WriteLine("Возврат разрешен. Применена скидка 50.");
        }
    }

    public void GetCheck()
    {
        
    }
}

// Создание товарной фактуры
GoodsInvoice goodsInvoice = new GoodsInvoice("INV001", DateTime.Now, DateTime.Now.AddDays(7));

// Добавление первого товара в фактуру
goodsInvoice.AddLine(new LineItem("Товар 1", 100, 2));

// Добавление второго товара в фактуру
goodsInvoice.AddLine(new LineItem("Товар 2", 200, 3));

// Вывод общей суммы по фактуре
Console.WriteLine($"Общая сумма по товарной фактуре: {goodsInvoice.TotalAmount}");



// Создание услуговой фактуры
ServiceInvoice serviceInvoice = new ServiceInvoice("INV002", DateTime.Now, DateTime.Now);

// Добавление услуги в фактуру
serviceInvoice.AddLine(new LineItem("Услуга 1", 300, 1));

// Удаление услуги из фактуры (аннулирование)
serviceInvoice.RemoveLine(new LineItem("Услуга 1", 300, 1));

// Вывод общей суммы
Console.WriteLine($"Общая сумма по услуговой фактуре: {serviceInvoice.TotalAmount}\n\n\n\n");



// Создание комбинированной фактуры
CombinedInvoice combinedInvoice = new CombinedInvoice("INV003", DateTime.Now, true);

// Добавляем строки в фактуру
combinedInvoice.AddLine(new LineItem("Товар 3", 400, 1));
combinedInvoice.AddLine(new LineItem("Услуга 2", 500, 1));

// Рассчитываем общую сумму с учетом возможного возврат
combinedInvoice.CalculateTotal();

// Выводим итоговую сумму
Console.WriteLine($"Общая сумма по комбинированной фактуре: {combinedInvoice.TotalAmount}");

Error: (204,29): error CS1729: "LineItem" не содержит конструктор, который принимает аргументы 4.